In [11]:
#base imports
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import RIGHT_ONLY

In [13]:
#test enviroment
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0', render_mode='human', apply_api_compatibility=True)
env = JoypadSpace(env, RIGHT_ONLY)
print(RIGHT_ONLY)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


[['NOOP'], ['right'], ['right', 'A'], ['right', 'B'], ['right', 'A', 'B']]


In [7]:
#test enviroment
env.observation_space.shape
env.action_space
SIMPLE_MOVEMENT

NameError: name 'SIMPLE_MOVEMENT' is not defined

In [3]:
#random choice agent
done = True
for frame in range(10000):
    if done:
        env.reset()
    state, reward, done, truncated, info = env.step(env.action_space.sample())
    print(state.shape)
env.close()


2024-10-22 22:01:23.674 Python[90331:7363061] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/x9/y984rskx5c15dyd3fq17kfn00000gq/T/org.python.python.savedState


(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 

KeyboardInterrupt: 

: 

In [12]:
#state = env.reset()
#env.step(1)[0 is state, 1 is reward, 2 whether we're dead/game, 3 is coins/flag,lives,score,stage,status]

import numpy as np
from gym import Wrapper
from gym.wrappers import FrameStack, GrayScaleObservation, ResizeObservation
from matplotlib import pyplot as plt

#skipframe wrapper(custom) skips 3 frames and uses the 4th for every frame in enviroment
class SkipFrame(Wrapper):
    def __init__(self, env, skip):
        super().__init__(env)
        self.skip = skip
    def step(self, action):
        total_reward = 0.0
        done = False
        for _ in range(self.skip):
            next_state,reward,done,trunc,info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return next_state, total_reward, done, trunc, info
    
def apply_wrappers(env):
    #our skipframe from earlier
     env = SkipFrame(env, skip=4)
     #resizes image to (84,84,3)
     env = ResizeObservation(env, shape = 84)
     #Grayscales image to (84,84)
     env = GrayScaleObservation(env)
     #stacks the frames to (4,84,84)
     env = FrameStack(env, num_stack=4,lz4_compress=True)
     return env





In [13]:
import torch
from torch import nn

#The neural network for the agent
class AgentNN(nn.Module):
    def __init__(self, input_shape, n_actions, freeze=False):
        super().__init__()
        #Layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8,stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        conv_out_size = self._get_conv_out(input_shape)

        #finalizing the output layers
        self.network = nn.Sequential(
            self.conv_layers,
            nn.Flatten(),
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
        if freeze:
            self._freeze()
        
        #uses gpu or cpu(in my case cpu)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.to(self.device)
    
    #gets the layers and shape of the network
    def _get_conv_out(self, shape):
        o = self.conv_layers(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
    
    #Disables gradients in network(for target)
    def _freeze(self):
        for p in self.network.parameters():
            p.requires_grad = False
            
    def forward(self, x):
        return self.network(x)

In [15]:
from tensordict import TensorDict
from torchrl.data import TensorDictReplayBuffer, LazyMemmapStorage

#agent class(Mario)
class Agent:
    def __init__(self, input_dims, num_actions):
        self.num_actions = num_actions
        self.learn_step_counter = 0
        
        #BASE PARAMETERS
        self.lr = 0.00025
        self.gamma = 0.9
        self.epsilon = 0.9
        self.eps_decay = 0.999
        self.eps_min = 0.1
        self.batch_size = 32
        self.sync_network_rate = 10_000
        
        #Network and target network creation
        self.online_network = AgentNN(input_dims, num_actions)
        self.target_network = AgentNN(input_dims, num_actions, freeze = True)
        
        #optimizing the network
        self.optimizer = torch.optim.Adam(self.online_network.parameters(), lr = self.lr)
        self.loss = torch.nn.MSELoss()
        
        replay_buffer_capacity = 100_000
        storage = LazyMemmapStorage(replay_buffer_capacity)
        self.replay_buffer = TensorDictReplayBuffer(storage=storage)
        
    #choose_action method
    #chooses a random action based on epsilon or makes an actual choice(returns 0-4)
    def choose_action(self, observation):
        if np.random.random() < self.epsilon:
            return np.random.randint(self.num_actions)
        observation = torch.tensor(np.array(observation), dtype=torch.float32) \
            .unsqueeze(0) \
            .to(self.online_network.device)
        return self.online_network(observation).argmax().item()
    
    #decays the epsilon value over time for the epsilon greedy approach
    def decay_epsilon(self):
        self.epsilon = max(self.epsilon * self.eps_decay, self.eps_min)
    
    #stores the memory of the all of the enviroment states
    def store_in_memory(self, state, action, reward, next_state, done):
        self.replay_buffer.add(TensorDict({
            "state":torch.tensor(np.array(state), dtype=torch.float32),
            "action": torch.tensor(action),
            "reward": torch.tensor(reward),
            "next_state":torch.tensor(np.array(next_state), dtype=torch.float32),
            "done": torch.tensor(done)
        }, batch_size=[]))
        
    def sync_networks(self):
        if self.learn_step_counter % self.sync_network_rate == 0 and self.learn_step_counter > 0:
            self.target_network.load_state_dict(self.online_network.state_dict())
            
    #THE MOST IMPORTANT FUNCTION
    def learn(self):
        if len(self.replay_buffer) < self.batch_size:
            return
        
        self.sync_networks()
        self.optimizer.zero_grad()
        samples = self.replay_buffer.sample(self.batch_size).to(self.online_network.device)
        keys = ("state","action","reward", "next_state", "done")
        
        states, actions, rewards, next_states, dones = [samples[key] for key in keys]
        predicted_q_values = self.online_network(states)
        predicted_q_values = predicted_q_values[np.arange(self.batch_size), actions.squeeze()]
        
        target_q_values = self.target_network(next_states).max(dim=1)[0]
        target_q_values = rewards + self.gamma * target_q_values * (1-dones.float())
        
        loss = self.loss(predicted_q_values, target_q_values)
        loss.backward()
        self.optimizer.step()
        
        self.learn_step_counter += 1
        self.decay_epsilon()
        
    #Methods to save and load model
    def save_model(self, path):
        torch.save(self.online_network.state_dict(), path)
    def load_model(self, path):
        self.online_network.load_state_dict(torch.load(path))
        self.target_network.load_state_dict(torch.load(path))
        
        

In [ ]:
ENV_NAME = 'SuperMarioBros-1-1-v0'
NUM_OF_ITERATION = 20000
SAVEINT = 250

#creates an enviroment to train model given the agent

env = gym_super_mario_bros.make(ENV_NAME,render_mode='rgb_array',apply_api_compatibility=True)
env = JoypadSpace(env, RIGHT_ONLY)
env = apply_wrappers(env)

agent = Agent(input_dims = env.observation_space.shape, num_actions = env.action_space.n)
agent.load_model("/Users/26prakash/Desktop/CompSci/MLAI/Project_02_gAIme/Models/3_iter.pt")
print(env.observation_space.shape)
print(env.action_space.n)
#agent.load_model("/Users/26prakash/Desktop/CompSci/MLAI/Project_02_gAIme/Models/1000_iter.pt")

for i in range(NUM_OF_ITERATION):
    print("iter: ", i)
    done = False
    state, _ = env.reset()
    while not done:
        action = agent.choose_action(state)
        new_state, reward, done, truncated, info = env.step(action)
        env.render()
        agent.store_in_memory(state,action,reward,new_state,done)
        agent.learn()
        state = new_state
        #print(state.shape)
    # if (i + 1) % SAVEINT == 0:
    #     agent.save_model("/Users/26prakash/Desktop/CompSci/MLAI/Project_02_gAIme/Models/" + str(i+1) + "_iter.pt")
    print("Reward: " + str(reward))
env.close()
    
        

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


(4, 84, 84)
5
iter:  0


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


Reward: -6.0
iter:  1
Reward: -15.0
iter:  2
Reward: -15.0
iter:  3
Reward: -13.0
iter:  4
Reward: -14.0
iter:  5
Reward: -15.0
iter:  6
Reward: -12.0
iter:  7
Reward: -6.0
iter:  8
Reward: -14.0
iter:  9
Reward: -15.0
iter:  10
Reward: -9.0
iter:  11
Reward: -9.0
iter:  12
Reward: -12.0
iter:  13
Reward: -12.0
iter:  14
Reward: -10.0
iter:  15
Reward: -12.0
iter:  16
Reward: -15.0
iter:  17
Reward: -13.0
iter:  18
Reward: -6.0
iter:  19
Reward: -15.0
iter:  20
Reward: -9.0
iter:  21
Reward: -13.0
iter:  22
Reward: -12.0
iter:  23
Reward: -6.0
iter:  24
Reward: -13.0
iter:  25
Reward: -13.0
iter:  26
Reward: -7.0
iter:  27
Reward: -15.0
iter:  28
Reward: -15.0
iter:  29
Reward: -8.0
iter:  30
Reward: -7.0
iter:  31
Reward: -16.0
iter:  32
Reward: -15.0
iter:  33
Reward: -12.0
iter:  34
Reward: -15.0
iter:  35
Reward: -15.0
iter:  36
Reward: -8.0
iter:  37
Reward: -16.0
iter:  38
Reward: -16.0
iter:  39
Reward: -8.0
iter:  40
Reward: -15.0
iter:  41
Reward: -13.0
iter:  42
Reward: -16.0

In [10]:
agent.save_model("/Users/26prakash/Desktop/CompSci/MLAI/Project_02_gAIme/Models/3_iter.pt")


TypeError: Cannot interpret '84' as a data type